## Does weather play a factor in when deliveries occur

In [1]:
#Import Libraries
import os
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression,LogisticRegression
import seaborn as sns 
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


In [2]:
#load dataset into a dataframe and confirm values
df_raw = pd.read_csv('DataSets\\Savage_Daily_Ticket_Count_Weather_Export.csv')
df_raw.head(12)

,delivery_count_sum,is_above_average_delivery_day,log_ratio_to_average,delivery_date,delivery_weekday_name,is_midweek,DailyPrecipitation,DailySnowDepth,SnowOnGround,DailySnowfall,...,PriorDayPrecipitationAmt,Prior2DayPrecipitationAmt,PriorDayPrecipitationDiff,Prior2DayPrecipitationDiff,PriorDaySnowfallAmt,PriorDaySnowfallDiff,Precip_12AM-6AM,Precip_6AM-10AM,Precip_6AM-12PM,Precip_10AM-2PM
0,2,False,0.000000,2021-01-07,Thu,True,0.00,8,True,0.0,...,0.00,0.00,0.00,0.00,0.0,0.0,No Precip,No Precip,No Precip,No Precip
1,2,False,0.000000,2020-01-10,Fri,False,0.00,3,True,0.0,...,0.00,0.00,0.00,0.00,0.0,0.0,No Precip,No Precip,No Precip,No Precip
2,1,False,0.000000,2011-01-13,Thu,True,0.08,10,True,1.9,...,0.00,0.10,0.08,-0.02,0.0,1.9,No Precip,No Precip,Light Precip,Precip
3,1,False,0.000000,2011-01-24,Mon,False,0.01,12,True,0.1,...,0.00,0.00,0.01,0.01,0.0,0.1,No Precip,No Precip,No Precip,Light Precip
4,2,False,0.000000,2018-01-31,Wed,True,0.13,5,True,2.1,...,0.00,0.00,0.13,0.13,0.0,2.1,Precip,No Precip,No Precip,No Precip
5,1,False,0.000000,2013-02-06,Wed,True,0.04,5,True,0.7,...,0.05,0.06,-0.01,-0.02,1.2,-0.5,No Precip,No Precip,Light Precip,Light Precip
6,2,False,0.000000,2017-02-09,Thu,True,0.00,0,False,0.0,...,0.00,0.01,0.00,-0.01,0.0,0.0,No Precip,No Precip,No Precip,No Precip
7,1,False,0.000000,2020-02-13,Thu,True,0.00,9,True,0.0,...,0.00,0.00,0.00,0.00,0.0,0.0,No Precip,No Precip,No Precip,No Precip
8,12,False,0.000000,2016-02-25,Thu,True,0.00,0,False,0.0,...,0.00,0.04,0.00,-0.04,0.0,0.0,No Precip,No Precip,No Precip,No Precip
9,5,False,-0.903090,2013-02-26,Tue,True,0.00,6,True,0.0,...,0.00,0.00,0.00,0.00,0.0,0.0,No Precip,No Precip,No Precip,No Precip


Response varaibles are:  
delivery_count_sum -> total deliveries for the date

log_ratio_to_average -> ratio of today's deliveries to the average on that day for the last 11 years (log transformed to normallize)

is_above_average_delivery_day -> to try logistic regression:  True if greater than the average

In [3]:
#establish the working set
df = df_raw.copy()

#don't need the delivery date field for analysis nor the day name
df = df.drop(['delivery_date','delivery_weekday_name'],axis=1)

#set values to string for analysis
df['SnowOnGround'] = df['SnowOnGround'].astype(str)
df['is_midweek'] = df['is_midweek'].astype(str)


In [4]:
#Correlations have been checked in dataset building, but will put a matrix up for completeness
corr = df_raw.corr()
corr.style.background_gradient(cmap='coolwarm')

,delivery_count_sum,is_above_average_delivery_day,log_ratio_to_average,is_midweek,DailyPrecipitation,DailySnowDepth,SnowOnGround,DailySnowfall,PriorDayAverageDryBulbTemperatureDiff,PriorDayMaximumTemperatureDiff,PriorDayMinimumTemperatureDiff,PriorDayPrecipitationAmt,Prior2DayPrecipitationAmt,PriorDayPrecipitationDiff,Prior2DayPrecipitationDiff,PriorDaySnowfallAmt,PriorDaySnowfallDiff
delivery_count_sum,1.000000,0.699812,0.684540,0.033508,0.013602,-0.180256,-0.209841,-0.092842,0.017620,0.013661,0.018282,0.015692,0.056086,-0.002015,-0.030033,-0.093728,-0.005833
is_above_average_delivery_day,0.699812,1.000000,0.715765,0.039456,-0.012871,-0.107103,-0.104156,-0.067173,0.002460,-0.003605,0.008667,-0.011272,0.006752,-0.000803,-0.013905,-0.056636,-0.012099
log_ratio_to_average,0.684540,0.715765,1.000000,0.010904,-0.026159,-0.140165,-0.137056,-0.085994,0.002544,-0.000205,0.007628,-0.036993,-0.002928,0.009038,-0.016482,-0.111550,0.012032
is_midweek,0.033508,0.039456,0.010904,1.000000,0.042467,0.033734,0.039789,0.010758,-0.031163,-0.045758,-0.002174,0.011202,0.008966,0.022340,0.023776,0.010489,0.000938
DailyPrecipitation,0.013602,-0.012871,-0.026159,0.042467,1.000000,-0.046065,-0.053277,0.129491,-0.108592,-0.148424,-0.016733,0.091786,0.003535,0.656371,0.706736,-0.023907,0.117129
DailySnowDepth,-0.180256,-0.107103,-0.140165,0.033734,-0.046065,1.000000,0.774955,0.160079,0.038992,0.028297,0.042128,-0.018210,-0.024930,-0.019643,-0.015033,0.375478,-0.140689
SnowOnGround,-0.209841,-0.104156,-0.137056,0.039789,-0.053277,0.774955,1.000000,0.238799,0.030078,0.020095,0.032926,-0.024059,-0.025300,-0.020447,-0.019887,0.374648,-0.079143
DailySnowfall,-0.092842,-0.067173,-0.085994,0.010758,0.129491,0.160079,0.238799,1.000000,-0.065219,-0.090589,-0.007391,-0.007207,-0.029462,0.099459,0.112684,0.088407,0.712090
PriorDayAverageDryBulbTemperatureDiff,0.017620,0.002460,0.002544,-0.031163,-0.108592,0.038992,0.030078,-0.065219,1.000000,0.897599,0.805128,-0.086567,-0.013921,-0.013242,-0.067169,-0.050120,-0.015179
PriorDayMaximumTemperatureDiff,0.013661,-0.003605,-0.000205,-0.045758,-0.148424,0.028297,0.020095,-0.090589,0.897599,1.000000,0.466969,-0.042807,0.026282,-0.075310,-0.123862,-0.022967,-0.053964


In [5]:
#function to handle multi-collinearity tests
def vif_calc(X):
    vif_info = pd.DataFrame()
    vif_info['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    vif_info['Column'] = X.columns
    vif_info.sort_values('VIF', ascending=False)
    return(vif_info)

#function to pass back AIC for linear model

def aic_calc(X,Y):
    #add constant to predictor variables
    X = sm.add_constant(X)
    #fit regression model
    model = sm.OLS(Y, X).fit()
    return(model.aic)


In [6]:
#create data sets for analysis, keep x's separate in case I change individual tests first is regression against the delivery count
xc = df.drop(['delivery_count_sum','is_above_average_delivery_day','log_ratio_to_average'],axis=1)
xc = pd.get_dummies(xc,drop_first = True)                #make dummies for categorical values for analysis
yc = df['delivery_count_sum']

In [7]:
#next, the ratio calc
xr = df.drop(['delivery_count_sum','is_above_average_delivery_day','log_ratio_to_average'],axis=1)
xr = pd.get_dummies(xr,drop_first = True)                #make dummies for categorical values for analysis
yr = df['log_ratio_to_average']

#finally, the log
xl = df.drop(['delivery_count_sum','is_above_average_delivery_day','log_ratio_to_average'],axis=1)
xl = pd.get_dummies(xl,drop_first = True)                #make dummies for categorical values for analysis
yl = df['is_above_average_delivery_day']

In [8]:
vif_calc(xc)

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


,VIF,Column
0,inf,DailyPrecipitation
1,2.696621,DailySnowDepth
2,inf,DailySnowfall
3,265.041789,PriorDayAverageDryBulbTemperatureDiff
4,119.189091,PriorDayMaximumTemperatureDiff
5,65.842482,PriorDayMinimumTemperatureDiff
6,inf,PriorDayPrecipitationAmt
7,inf,Prior2DayPrecipitationAmt
8,inf,PriorDayPrecipitationDiff
9,inf,Prior2DayPrecipitationDiff


This data (as expected) has columns that have correlations.  In general the "Diff" fields are better distributed than the "Amt" so we will remove and run this again.  Also remove the "no precip" fields since they have high correlations.

In [9]:
xc = xc.drop(['PriorDayPrecipitationAmt','Prior2DayPrecipitationAmt','PriorDaySnowfallAmt','Precip_12AM-6AM_No Precip','Precip_6AM-10AM_No Precip','Precip_6AM-12PM_No Precip','Precip_10AM-2PM_No Precip'],axis=1)
vif_calc(xc)

,VIF,Column
0,3.688010,DailyPrecipitation
1,2.691790,DailySnowDepth
2,2.602310,DailySnowfall
3,264.790860,PriorDayAverageDryBulbTemperatureDiff
4,119.023235,PriorDayMaximumTemperatureDiff
5,65.791737,PriorDayMinimumTemperatureDiff
6,1.812148,PriorDayPrecipitationDiff
7,1.974165,Prior2DayPrecipitationDiff
8,2.454478,PriorDaySnowfallDiff
9,1.247900,is_midweek_True


All looking good except need to pick which Temperature Diff field to keep. Check for lowest AIC from keeping 1 of them.

In [10]:
aic_val =np.empty(4)
aic_val[0] = aic_calc(xc,yc)
temp_x = xc.drop(['PriorDayMaximumTemperatureDiff','PriorDayMinimumTemperatureDiff'],axis=1)
aic_val[1] = aic_calc(temp_x,yc)
temp_x = xc.drop(['PriorDayMinimumTemperatureDiff','PriorDayAverageDryBulbTemperatureDiff'],axis=1)
aic_val[2] = aic_calc(temp_x,yc)
temp_x = xc.drop(['PriorDayMaximumTemperatureDiff','PriorDayAverageDryBulbTemperatureDiff'],axis=1)
aic_val[3] = aic_calc(temp_x,yc)
print(aic_val)




[27572.1525601  27568.91203952 27569.73295657 27568.29817082]


C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  x = pd.concat(x[::order], 1)


In [11]:
xc = xc.drop(['PriorDayMaximumTemperatureDiff','PriorDayAverageDryBulbTemperatureDiff'],axis=1)
vif_calc(xc)

,VIF,Column
0,3.685394,DailyPrecipitation
1,2.690219,DailySnowDepth
2,2.594989,DailySnowfall
3,1.026933,PriorDayMinimumTemperatureDiff
4,1.811045,PriorDayPrecipitationDiff
5,1.963905,Prior2DayPrecipitationDiff
6,2.453428,PriorDaySnowfallDiff
7,1.246875,is_midweek_True
8,2.843407,SnowOnGround_True
9,1.395769,Precip_12AM-6AM_Precip


In [12]:
#winner!  remove columns and do same aic check on the other columns.
xr = xr.drop(['PriorDayPrecipitationAmt','Prior2DayPrecipitationAmt','PriorDaySnowfallAmt','Precip_12AM-6AM_No Precip','Precip_6AM-10AM_No Precip','Precip_6AM-12PM_No Precip','Precip_10AM-2PM_No Precip'],axis=1)
vif_calc(xr)

aic_val =np.empty(4)
aic_val[0] = aic_calc(xr,yr)
temp_x = xr.drop(['PriorDayMaximumTemperatureDiff','PriorDayMinimumTemperatureDiff'],axis=1)
aic_val[1] = aic_calc(temp_x,yr)
temp_x = xr.drop(['PriorDayMinimumTemperatureDiff','PriorDayAverageDryBulbTemperatureDiff'],axis=1)
aic_val[2] = aic_calc(temp_x,yr)
temp_x = xr.drop(['PriorDayMaximumTemperatureDiff','PriorDayAverageDryBulbTemperatureDiff'],axis=1)
aic_val[3] = aic_calc(temp_x,yr)
print(aic_val)

xr = xr.drop(['PriorDayMinimumTemperatureDiff','PriorDayAverageDryBulbTemperatureDiff'],axis=1)
vif_calc(xr)

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  x = pd.concat(x[::order], 1)


[3260.91960117 3258.25869181 3258.13880788 3258.28616675]


C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  x = pd.concat(x[::order], 1)


,VIF,Column
0,3.674388,DailyPrecipitation
1,2.687894,DailySnowDepth
2,2.596317,DailySnowfall
3,1.051683,PriorDayMaximumTemperatureDiff
4,1.796038,PriorDayPrecipitationDiff
5,1.962646,Prior2DayPrecipitationDiff
6,2.441150,PriorDaySnowfallDiff
7,1.246911,is_midweek_True
8,2.843687,SnowOnGround_True
9,1.396611,Precip_12AM-6AM_Precip


In [13]:
xl = xl.drop(['PriorDayPrecipitationAmt','Prior2DayPrecipitationAmt','PriorDaySnowfallAmt','Precip_12AM-6AM_No Precip','Precip_6AM-10AM_No Precip','Precip_6AM-12PM_No Precip','Precip_10AM-2PM_No Precip'],axis=1)
vif_calc(xl)

aic_val =np.empty(4)
aic_val[0] = aic_calc(xl,yl)
temp_x = xl.drop(['PriorDayMaximumTemperatureDiff','PriorDayMinimumTemperatureDiff'],axis=1)
aic_val[1] = aic_calc(temp_x,yl)
temp_x = xl.drop(['PriorDayMinimumTemperatureDiff','PriorDayAverageDryBulbTemperatureDiff'],axis=1)
aic_val[2] = aic_calc(temp_x,yl)
temp_x = xl.drop(['PriorDayMaximumTemperatureDiff','PriorDayAverageDryBulbTemperatureDiff'],axis=1)
aic_val[3] = aic_calc(temp_x,yl)
print(aic_val)

xl = xl.drop(['PriorDayMaximumTemperatureDiff','PriorDayAverageDryBulbTemperatureDiff'],axis=1)
vif_calc(xl)

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  x = pd.concat(x[::order], 1)


[3171.86520267 3168.76147696 3168.59722688 3168.52102732]


C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  x = pd.concat(x[::order], 1)


,VIF,Column
0,3.685394,DailyPrecipitation
1,2.690219,DailySnowDepth
2,2.594989,DailySnowfall
3,1.026933,PriorDayMinimumTemperatureDiff
4,1.811045,PriorDayPrecipitationDiff
5,1.963905,Prior2DayPrecipitationDiff
6,2.453428,PriorDaySnowfallDiff
7,1.246875,is_midweek_True
8,2.843407,SnowOnGround_True
9,1.395769,Precip_12AM-6AM_Precip


I now have 3 datasets with very strong VIF stats.  Time to run some linear regression to get a baseline

In [14]:
#function to run a linear regression
def get_stats(x,y,log=False):
    if (log == True):
        results = sm.Logit(y,x).fit()
    else: results = sm.OLS(y, x).fit()
    print(results.summary())

get_stats(xr,yr) 
#results show precip 6-10 and 6-12 are not significant so remove and try again
temp_xr = xr.drop(['Precip_6AM-10AM_Precip','Precip_6AM-12PM_Precip'],axis=1)
get_stats(temp_xr,yr) 

#remove the other non-sginifcant variables
temp_xr = xr.drop(['PriorDayMaximumTemperatureDiff','Precip_6AM-10AM_Precip','Precip_6AM-12PM_Precip','Precip_12AM-6AM_Precip','Precip_10AM-2PM_Precip'],axis=1)
get_stats(temp_xr,yr) 


                                  OLS Regression Results                                 
Dep. Variable:     log_ratio_to_average   R-squared (uncentered):                   0.126
Model:                              OLS   Adj. R-squared (uncentered):              0.121
Method:                   Least Squares   F-statistic:                              24.64
Date:                  Sun, 27 Mar 2022   Prob (F-statistic):                    2.44e-56
Time:                          18:19:50   Log-Likelihood:                         -1668.8
No. Observations:                  2235   AIC:                                      3364.
Df Residuals:                      2222   BIC:                                      3438.
Df Model:                            13                                                  
Covariance Type:              nonrobust                                                  
                                     coef    std err          t      P>|t|      [0.025      0.975]
-

An adjusted R^2 of .122 is not much to work with for the ratio

In [15]:
get_stats(xc,yc) 
temp_xc = xc.drop(['PriorDaySnowfallDiff','Precip_10AM-2PM_Precip'],axis=1)
get_stats(temp_xc,yc) 

temp_xc = temp_xc.drop(['DailySnowDepth','DailySnowfall','Precip_6AM-10AM_Precip','Precip_6AM-12PM_Precip'],axis=1)
get_stats(temp_xc,yc) 



                                 OLS Regression Results                                
Dep. Variable:     delivery_count_sum   R-squared (uncentered):                   0.363
Model:                            OLS   Adj. R-squared (uncentered):              0.359
Method:                 Least Squares   F-statistic:                              97.33
Date:                Sun, 27 Mar 2022   Prob (F-statistic):                   2.56e-206
Time:                        18:19:50   Log-Likelihood:                         -14171.
No. Observations:                2235   AIC:                                  2.837e+04
Df Residuals:                    2222   BIC:                                  2.844e+04
Df Model:                          13                                                  
Covariance Type:            nonrobust                                                  
                                     coef    std err          t      P>|t|      [0.025      0.975]
---------------------

For the daily count, it is a bit more promising with an Adjusted R^2 = 0.359

In [16]:
#now look at logistic Regression
get_stats(xl,yl,log=True)

#start dropping the highest
temp_xl = xl.drop(['PriorDayMinimumTemperatureDiff','Precip_10AM-2PM_Precip','Precip_6AM-10AM_Precip','PriorDaySnowfallDiff'],axis=1)
get_stats(temp_xl,yl,log=True)

Optimization terminated successfully.
         Current function value: 0.674332
         Iterations 6
                                 Logit Regression Results                                
Dep. Variable:     is_above_average_delivery_day   No. Observations:                 2235
Model:                                     Logit   Df Residuals:                     2222
Method:                                      MLE   Df Model:                           12
Date:                           Sun, 27 Mar 2022   Pseudo R-squ.:                0.007974
Time:                                   18:19:50   Log-Likelihood:                -1507.1
converged:                                  True   LL-Null:                       -1519.2
Covariance Type:                       nonrobust   LLR p-value:                   0.01893
                                     coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------

In [17]:
#start dropping the highest
temp_xl = temp_xl.drop(['Prior2DayPrecipitationDiff','SnowOnGround_True','Precip_12AM-6AM_Precip','Precip_6AM-12PM_Precip'],axis=1)
get_stats(temp_xl,yl,log=True)

Optimization terminated successfully.
         Current function value: 0.675755
         Iterations 6
                                 Logit Regression Results                                
Dep. Variable:     is_above_average_delivery_day   No. Observations:                 2235
Model:                                     Logit   Df Residuals:                     2230
Method:                                      MLE   Df Model:                            4
Date:                           Sun, 27 Mar 2022   Pseudo R-squ.:                0.005880
Time:                                   18:19:50   Log-Likelihood:                -1510.3
converged:                                  True   LL-Null:                       -1519.2
Covariance Type:                       nonrobust   LLR p-value:                  0.001311
                                coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------

The 3 features in common with all models are:  DailyPrecipitation, PriorDayPrecipitationDiff, is_midweek_True

In [18]:
#now the logistic Regression
log_reg = LogisticRegression()
log_mod = log_reg.fit(temp_xl,yl)
print(log_mod.score(temp_xl,yl))       #highly overfitted
#y_pred = log_reg.predict(x)

0.5816554809843401


In [ ]:
def run_lasso(X,Y,graph=False,iter_nbr=1000,a_start = .01,a_stop = 1,a_step=.01):
    #X = predictors, Y = response, splits = cv splits
    x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=.24,random_state=5440)
    cv = RepeatedKFold(n_splits = 10,n_repeats=50, random_state=5440)   #set random_state to make results repeatable
    model = make_pipeline(StandardScaler(),LassoCV(alphas=np.arange(a_start,a_stop,a_step),cv=cv,tol = .005,max_iter=iter_nbr,n_jobs=-1))
    
    #fit the model on all the data to give us the optimal "alpha/lambda value"
    model.fit(X,Y)
      
    #Now, use the training and test sets to get the scoring
    pred_train = model.predict(x_train)
    pred_test = model.predict(x_test)
    rmse_train = np.sqrt(mean_squared_error(y_train,pred_train))
    r2_train = r2_score(y_train, pred_train)
    rmse_test = np.sqrt(mean_squared_error(y_test,pred_test))
    r2_test = r2_score(y_test, pred_test)  
    
    if graph:
        graph_result(x_test,y_test,pred_test)
    
    return(rmse_test,r2_test)

def run_elastic_net(X,Y,graph=False,scaling_list = [.1,.2,.3,.4,.5,.6,.7,.8,.9],iter_nbr=1000):
    #X = predictors, Y = response, scaling_list = the alpha values between 0 and 1, splits = cv splits
    x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=.24,random_state=5440)
    cv = RepeatedKFold(n_splits = 10,n_repeats=50, random_state=5440)   #set random_state to make results repeatable
    model = make_pipeline(StandardScaler(),ElasticNetCV(alphas=scaling_list,cv=cv,tol = .005,max_iter=iter_nbr,n_jobs=-1))
    
    #fit the model on all the data to give us the optimal "alpha/lambda value"
    model.fit(X,Y)
    
    #Now, use the training and test sets to get the scoring
    pred_train = model.predict(x_train)
    pred_test = model.predict(x_test)
    rmse_train = np.sqrt(mean_squared_error(y_train,pred_train))
    r2_train = r2_score(y_train, pred_train)
    rmse_test = np.sqrt(mean_squared_error(y_test,pred_test))
    r2_test = r2_score(y_test, pred_test)  
    
    if graph:
        graph_result(x_test,y_test,pred_test)
        
    return(rmse_test,r2_test)

def run_ridge(X,Y,graph=False,a_start = 5,a_stop = 15,a_step=.1):
    #X = predictors, Y = response, other numbers for the range of values
    x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=.24,random_state=5440)
    cv = RepeatedKFold(n_splits = 10,n_repeats=50,  random_state=5440)   #set random_state to make results repeatable
    model = make_pipeline(StandardScaler(),RidgeCV(alphas=np.arange(a_start,a_stop,a_step),cv=cv))
    
    #fit the model on all the data to give us the optimal "alpha/lambda value"
    model.fit(X,Y)
     
    #Now, use the training and test sets to get the scoring
    pred_train = model.predict(x_train)
    pred_test = model.predict(x_test)
    rmse_train = np.sqrt(mean_squared_error(y_train,pred_train))
    r2_train = r2_score(y_train, pred_train)
    rmse_test = np.sqrt(mean_squared_error(y_test,pred_test))
    r2_test = r2_score(y_test, pred_test)  
    
    if graph:
        graph_result(x_test,y_test,pred_test)
    
    return(rmse_test,r2_test)


In [ ]:

def run_ridge_classification(X,Y,graph=False,a_start = 5,a_stop = 15,a_step=.1):
    #X = predictors, Y = response, other numbers for the range of values
    x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=.24,random_state=5440)
    cv = RepeatedStratifiedKFold(n_splits = 10,n_repeats=50,  random_state=5440)   #set random_state to make results repeatable
    model = make_pipeline(StandardScaler(),RidgeClassifierCV(alphas=np.arange(a_start,a_stop,a_step),cv=cv))
    
    #fit the model on all the data to give us the optimal "alpha/lambda value"
    model.fit(X,Y)
     
    #find the worth of the model
    pred_train = model.predict(x_train)
    train_score = model.score(x_train,y_train)   
    pred_test = model.predict(x_test)
    test_score = model.score(x_test,y_test)
    
    if graph:
        cm = confusion_matrix(y_test,pred_test)
        disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=model.classes_)
        disp.plot(cmap='BuGn')
        plt.show()

    return(test_score)


def run_QDA_classification(X,Y,graph=False):
    #X = predictors, Y = response, other numbers for the range of values
    x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=.24,random_state=5440)
    cv = RepeatedStratifiedKFold(n_splits = 10,n_repeats=50,  random_state=5440)   #set random_state to make results repeatable
    model = make_pipeline(StandardScaler(),QuadraticDiscriminantAnalysis())
    
    #fit the model on all the data to give us the optimal "alpha/lambda value"
    #model.fit(X,Y)
     
    #find the worth of the model
    #pred_train = model.predict(x_train)
    #train_score = model.score(x_train,y_train)   
    #pred_test = model.predict(x_test)
    #test_score = model.score(x_test,y_test)
    
    test_score = cross_val_predict(model,X,Y,cv=10,n_jobs=-1)
    
    if graph:
        cm = confusion_matrix(Y,test_score)
        disp = ConfusionMatrixDisplay(confusion_matrix=cm)
        disp.plot(cmap='BuGn')
        plt.show()

    return(test_score.mean())